In [1]:
import io
import itertools

import numpy as np
import sklearn.metrics
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

C:\Users\Arsene\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
#loading the data
train_data = np.load(r'datasets/Shoes - All - Train.npz')
val_data = np.load(r'datasets/Shoes - All - Validation.npz')
test_data = np.load(r'datasets/Shoes - All - Test.npz')

In [3]:
train_images, train_labels = train_data['images'], train_data['labels']
val_images, val_labels = val_data['images'], val_data['labels']
test_images, test_labels = test_data['images'], test_data['labels']

In [4]:
#feature scaling
train_images = train_images/255.
val_images = val_images/255.
test_images = test_images/255.

In [5]:
#constant definition
NUM_EPOCHS = 11
BATCH_SIZE = 64

In [6]:
#hyperparam definition
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([5,7]))
HP_NUM_FILTER = hp.HParam('filters_number', hp.Discrete([96,128]))
HP_DENSE_SIZE = hp.HParam('dense_size', hp.Discrete([512,1024]))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer(r'D:/sam/Logs/Model 5/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams= [HP_FILTER_SIZE,HP_NUM_FILTER,HP_DENSE_SIZE],
        metrics = [hp.Metric(METRIC_ACCURACY,display_name='accuracy')]
    )

In [7]:
#model definition and training
def train_test_model(hparams,session_num):
    
    #model outline
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_NUM_FILTER],hparams[HP_FILTER_SIZE], activation='relu', input_shape=(120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_NUM_FILTER],5,activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_DENSE_SIZE], activation='relu'),
        tf.keras.layers.Dense(11)
    ])
    
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
    model.compile(optimizer='adam', loss= loss_fn, metrics=['accuracy'])
    
    #defining the logging dir
    log_dir = 'D:/sam/Logs/Model 5/fit/'+'run-{}'.format(session_num)
    
    def plot_confusion_matrix(cm, class_names):
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label') 
        return figure

    def plot_to_image(figure):
    
        #save the plot to a png in memory
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
    
        #closing the figure to prevent from displaying in the notebook
        plt.close(figure)
    
        buf.seek(0)
    
        image = tf.image.decode_png(buf.getvalue(), channels=4)
    
        #add the batch dimension
        image = tf.expand_dims(image, 0)
    
        return image

    #file writer for logging cofusion matrix
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')
    
    def log_confusion_matrix(epoch, logs):
    #predict values for validation set
        test_pred_r = model.predict(val_images)
        test_pred = np.argmax(test_pred_r, axis=1)
    
        cm = sklearn.metrics.confusion_matrix(val_labels, test_pred)
    
        figure = plot_confusion_matrix(cm, class_names=[ 'Male Boots', 'Male Trainers/sneakers','Male Sandals',
                                                        'Male Formal shoes', 'Male Other','Female Boots', 'Female Ballerina shoes','Female Sneakers/trainesr',
                                                        'Female Sandals','Female High heels','Female Other'])
        cm_image = plot_to_image(figure)
    
        #log the confusion matrix as image summary
        with file_writer_cm.as_default():
            tf.summary.image('Confusion Matrix', cm_image, step=epoch)
            
    
    
    #defining callbacks
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq = 1, profile_batch = 0)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end= log_confusion_matrix)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode = 'auto',
        verbose=0,
        patience = 2,
        min_delta = 0,
        restore_best_weights= True
    )

    
    #training the model
    model.fit(
        train_images,
        train_labels,
        validation_data= (val_images,val_labels),
        verbose=2,
        epochs= NUM_EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks=[tensorboard_callback,cm_callback,early_stopping]
        
    )
    
    #model performance evaluation
    _, accuracy = model.evaluate(val_images,val_labels)
    
    #saving the model
    model.save(r'D:/sam/saved Models/Model 5/Run-{}'.format(session_num))
    
    return accuracy


In [8]:
#logging the result function
def run(log_dir, hparams, session_num):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)


In [9]:
session_num = 1

for filter_size in HP_FILTER_SIZE.domain.values:
    for num_filter in HP_NUM_FILTER.domain.values:
        for dense_size in HP_DENSE_SIZE.domain.values:
        
            hparams = {
                HP_FILTER_SIZE : filter_size,
                HP_NUM_FILTER : num_filter,
                HP_DENSE_SIZE: dense_size}

            run_name = 'run-%d' %session_num
            print('---- Starting trial : %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('D:/sam/Logs/Model 5/hparam_tuning/' +run_name, hparams, session_num)

            session_num += 1

---- Starting trial : run-1
{'filter_size': 5, 'filters_number': 96, 'dense_size': 512}
Epoch 1/11
78/78 - 140s - loss: 1.6918 - accuracy: 0.4882 - val_loss: 1.3169 - val_accuracy: 0.5267 - 140s/epoch - 2s/step
Epoch 2/11
78/78 - 136s - loss: 0.9904 - accuracy: 0.6647 - val_loss: 1.1173 - val_accuracy: 0.6591 - 136s/epoch - 2s/step
Epoch 3/11
78/78 - 136s - loss: 0.8521 - accuracy: 0.7096 - val_loss: 1.0843 - val_accuracy: 0.6527 - 136s/epoch - 2s/step
Epoch 4/11
78/78 - 141s - loss: 0.8370 - accuracy: 0.7072 - val_loss: 0.9627 - val_accuracy: 0.7108 - 141s/epoch - 2s/step
Epoch 5/11
78/78 - 133s - loss: 0.6298 - accuracy: 0.7766 - val_loss: 1.0718 - val_accuracy: 0.6640 - 133s/epoch - 2s/step
Epoch 6/11
78/78 - 129s - loss: 0.5383 - accuracy: 0.8078 - val_loss: 1.0070 - val_accuracy: 0.6882 - 129s/epoch - 2s/step
20/20 [==============================] - 5s 237ms/step - loss: 0.9627 - accuracy: 0.7108
INFO:tensorflow:Assets written to: D:/sam/saved Models/Model 5/Run-1\assets
---- Star

Epoch 3/11
78/78 - 649s - loss: 0.9536 - accuracy: 0.6856 - val_loss: 1.1405 - val_accuracy: 0.6123 - 649s/epoch - 8s/step
Epoch 4/11
78/78 - 383s - loss: 0.8777 - accuracy: 0.7054 - val_loss: 1.0670 - val_accuracy: 0.6462 - 383s/epoch - 5s/step
Epoch 5/11
78/78 - 381s - loss: 0.8333 - accuracy: 0.7177 - val_loss: 0.9281 - val_accuracy: 0.6882 - 381s/epoch - 5s/step
Epoch 6/11
78/78 - 771s - loss: 0.7258 - accuracy: 0.7435 - val_loss: 0.8781 - val_accuracy: 0.7076 - 771s/epoch - 10s/step
Epoch 7/11
78/78 - 450s - loss: 0.6669 - accuracy: 0.7623 - val_loss: 0.8656 - val_accuracy: 0.7060 - 450s/epoch - 6s/step
Epoch 8/11
78/78 - 551s - loss: 0.5930 - accuracy: 0.7818 - val_loss: 0.8573 - val_accuracy: 0.7173 - 551s/epoch - 7s/step
Epoch 9/11
78/78 - 623s - loss: 0.5303 - accuracy: 0.8052 - val_loss: 0.9505 - val_accuracy: 0.7027 - 623s/epoch - 8s/step
Epoch 10/11
78/78 - 698s - loss: 0.5050 - accuracy: 0.8149 - val_loss: 0.9157 - val_accuracy: 0.7496 - 698s/epoch - 9s/step
20/20 [=======

In [10]:
# Loading a model to evaluate on the test set
model = tf.keras.models.load_model(r"D:/sam/saved Models/Model 5/Run-4")

In [11]:
test_loss, test_accuracy = model.evaluate(test_images,test_labels)

20/20 [==============================] - 7s 294ms/step - loss: 0.7423 - accuracy: 0.7254


In [12]:
# Printing the test results
print('Test loss: {0:.4f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.7423. Test accuracy: 72.54%


In [13]:
%load_ext tensorboard
%tensorboard --logdir "D:/sam/Logs/Model 5/hparam_tuning"

In [15]:
%load_ext tensorboard
%tensorboard --logdir "D:/sam/Logs/Model 5/fit"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
